In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt

# Kiểm tra và cấu hình GPU
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Cấu hình GPU để tận dụng tối đa bộ nhớ
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. Running on CPU")

# 1. Tải và tiền xử lý dữ liệu
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Chuẩn hóa dữ liệu về khoảng [0,1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Chuyển nhãn sang one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# 2. Xây dựng mô hình CNN
model = models.Sequential([
    # Khối Convolution thứ 1
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    
    # Khối Convolution thứ 2
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    
    # Khối Fully Connected
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

# 3. Biên dịch mô hình với tối ưu hóa GPU
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    jit_compile=True  # Kích hoạt XLA compilation
)

# In tóm tắt kiến trúc mô hình
model.summary()

# 4. Huấn luyện mô hình với tối ưu hóa GPU
history = model.fit(
    x_train, y_train,
    batch_size=128,  # Tăng batch size để tận dụng GPU
    epochs=30,
    validation_split=0.2,
    verbose=1,
    workers=4,  # Số lượng worker cho data loading
    use_multiprocessing=True  # Kích hoạt multiprocessing
)

# 5. Đánh giá mô hình
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f'\nĐộ chính xác trên tập test: {test_acc*100:.2f}%')

# 6. Vẽ đồ thị quá trình huấn luyện
plt.figure(figsize=(12, 4))

# Đồ thị độ chính xác
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Độ chính xác (training)')
plt.plot(history.history['val_accuracy'], label='Độ chính xác (validation)')
plt.title('Độ chính xác qua các epoch')
plt.xlabel('Epoch')
plt.ylabel('Độ chính xác')
plt.legend()

# Đồ thị loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Loss (training)')
plt.plot(history.history['val_loss'], label='Loss (validation)')
plt.title('Loss qua các epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# 7. Thử nghiệm dự đoán một số ảnh với tối ưu hóa GPU
def predict_images(model, x_test, y_test, class_names, num_images=5):
    # Chọn ngẫu nhiên một số ảnh
    test_indices = np.random.randint(0, len(x_test), num_images) 
    
    plt.figure(figsize=(15, 3))
    for i, idx in enumerate(test_indices):
        plt.subplot(1, num_images, i+1)
        plt.imshow(x_test[idx])
        
        # Dự đoán với batch processing
        batch = np.expand_dims(x_test[idx], axis=0)
        prediction = model.predict(batch, verbose=0)
        predicted_class = class_names[np.argmax(prediction)]
        true_class = class_names[np.argmax(y_test[idx])]
        
        # Hiển thị kết quả
        title = f'Thực tế: {true_class}\nDự đoán: {predicted_class}'
        plt.title(title, color='green' if predicted_class == true_class else 'red')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Danh sách tên các lớp
class_names = ['Máy bay', 'Xe hơi', 'Chim', 'Mèo', 'Chó', 'Hươu', 'Ếch', 'Ngựa', 'Tàu', 'Xe tải']

# Thử nghiệm dự đoán
predict_images(model, x_test, y_test, class_names)

ModuleNotFoundError: No module named 'tensorflow.keras'